Inicio según la recomendación dada por * Henry * en el cual indican empezar con el ETL

In [112]:
#Importo las librerías que requiero
import pandas as pd
import ast

In [113]:
#Extraigo los archivos requeridos para el análisis de datos
df = pd.read_csv('credits.csv')
df_movie_csv = pd.read_csv('movies_dataset.csv')
df_movie1 = pd.read_excel('movies_dataset1.xlsx')
df_movie_excel = pd.read_excel('movies_dataset.xlsx')

C:\Users\Patricia\AppData\Local\Temp\ipykernel_20488\648635763.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movie_csv = pd.read_csv('movies_dataset.csv')


In [114]:
#Como existen 3 archviso que se llaman movies_datasets de los cuales 2 de ellos son archvos excel verifico si contienen los mismos datos
evaluacion_identidad = df_movie_csv.equals(df_movie1)
print(evaluacion_identidad)

False


In [115]:
#Otra forma de verificar si los datos son iguales. 
are_equal = (df_movie_excel == df_movie1).all().all()
print(are_equal)  # True


False


In [116]:
#De acuerdo con la metodología planteada por el empleador (cliente-Henry), se eliminan las columnas. 
#Para no repetir codigo utilizo un ciclo for que itere sobre cada dataframe
columnasEliminar = ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'] 

for df in [df_movie1, df_movie_csv, df_movie_excel]:
    df.drop(columns=columnasEliminar, inplace=True)

In [117]:
#El empleador solicita que normalicemos los datos aplicando 0 a los valores nulos de las columnas revenue y budget,
#para lo cual también utilizo un ciclo for.

for df in [df_movie1, df_movie_csv, df_movie_excel]:
    df['revenue'].fillna(0, inplace=True)
    df['budget'].fillna(0, inplace=True)

In [118]:
#Se me solicita que elimine los valores nulos asociados a la columna "release date", para ello nuevamento itero con un
#ciclo for
for df in [df_movie1, df_movie_csv, df_movie_excel]:
    df.dropna(subset=['release_date'], inplace=True)


In [119]:
#Garantizo que se hayan generado los cambios de eliminación de valores nulos
print(df_movie1['release_date'].isnull().sum())
print(df_movie_csv['release_date'].isnull().sum())
print(df_movie_excel['release_date'].isnull().sum())

0
0
0


In [120]:
#Adicionalmente el cliente solicita que se transformen los datos de "año de lanzamiento" en la columna "release year"
# y coloquemos directamente a formato año y generemos otra columna con el año de lanzamiento.

for df in [df_movie1, df_movie_csv, df_movie_excel]:
    df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d', errors='coerce')
    df['release_year'] = df['release_date'].dt.year.astype(float).fillna(0).astype(int)

    # Mostrar los resultados
    print(df[['release_date', 'release_year']].head(3))


  release_date  release_year
0   1995-10-30          1995
1   1995-12-15          1995
2   1995-12-22          1995
  release_date  release_year
0   1995-10-30          1995
1   1995-12-15          1995
2   1995-12-22          1995
  release_date  release_year
0   1995-10-30          1995
1   1995-12-15          1995
2   1995-12-22          1995


In [121]:
#Evalúo el tipo de dato 
print(df_movie1['release_date'].dtype)
print(df_movie_csv['release_date'].dtype)
print(df_movie_excel['release_date'].dtype)


print(df_movie1['release_year'].dtype)
print(df_movie_csv['release_year'].dtype)
print(df_movie_excel['release_year'].dtype)

datetime64[ns]
datetime64[ns]
datetime64[ns]
int32
int32
int32


In [122]:
#Se nos solicita que hagamos una columna con el retorno de ganancia "return" para ello debemos dividir ganancias "revenue"/inversión o presupuesto
#que seria la columna #budget#
for df in [df_movie1, df_movie_csv, df_movie_excel]:
    df['return'] = 0  # Inicializamos la columna return con ceros
    df['budget'] = pd.to_numeric(df['budget'], errors='coerce')
    df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
    for index, row in df.iterrows():
        if row['budget'] != 0 and row['revenue'] != 0:
            df.loc[index, 'return'] = row['revenue'] / row['budget']

In [123]:
#Se itera sobre cada uno de los DF para intentar desanidar, no se pudo hacer con bucle for porque generaba un error
#aparentemente el problema está en el documento csv

# Desanidar la columna 'belongs_to_collection'
df_movie1['belongs_to_collection'] = df_movie1['belongs_to_collection'].apply(lambda x: ast.literal_eval(x)['name'] if (pd.notna(x) and isinstance(x, str)) else '')

# Desanidar la columna 'production_companies'
df_movie1['production_companies'] = df_movie1['production_companies'].apply(lambda x: ', '.join([company['name'] for company in ast.literal_eval(x)]) if (pd.notna(x) and isinstance(x, str)) else '')

# Mostrar los resultados
print(df_movie1[['belongs_to_collection', 'production_companies']].head(3))



       belongs_to_collection   
0       Toy Story Collection  \
1                              
2  Grumpy Old Men Collection   

                                production_companies  
0                            Pixar Animation Studios  
1  TriStar Pictures, Teitler Film, Interscope Com...  
2                       Warner Bros., Lancaster Gate  


In [151]:
# Desanidar la columna 'belongs_to_collection'
df_movie_excel['belongs_to_collection'] = df_movie_excel['belongs_to_collection'].apply(lambda x: ast.literal_eval(x)['name'] if (pd.notna(x) and isinstance(x, str)) else '')

# Desanidar la columna 'production_companies'
df_movie_excel['production_companies'] = df_movie_excel['production_companies'].apply(lambda x: ', '.join([company['name'] for company in ast.literal_eval(x)]) if (pd.notna(x) and isinstance(x, str)) else '')

# Mostrar los resultados
print(df_movie_excel[['belongs_to_collection', 'production_companies']].head(3))

       belongs_to_collection   
0       Toy Story Collection  \
1                              
2  Grumpy Old Men Collection   

                                production_companies  
0                            Pixar Animation Studios  
1  TriStar Pictures, Teitler Film, Interscope Com...  
2                       Warner Bros., Lancaster Gate  


In [149]:
#Se evalúa el documento csv aparentemente existen datos vacíos que impiden leerse bien
def parse_json(value):
    try:
        if pd.notna(value) and isinstance(value, str) and value != '' and value != '[]':
            json_data = json.loads(value)
            if isinstance(json_data, dict) and 'name' in json_data:
                return json_data['name']
            elif isinstance(json_data, list):
                return ', '.join([company['name'] for company in json_data])
        return ''  # Agregar esta línea para manejar valores vacíos
    except Exception:
        return ''


In [150]:
print(df_movie_csv[['belongs_to_collection', 'production_companies']].head(3))


  belongs_to_collection production_companies
0                                           
1                                           
2                                           


Genero las funciones para hacer la API

In [156]:
#Según lo indicado por el cliente se genera una función que retorne la cantidad de películas estrenadas segun idioma y año
from fastapi import FastAPI

app = FastAPI()

@app.get("/peliculas_idioma")
def peliculas_idioma(idioma: str, año: int):
    total = 0
    for df in [df_movie1, df_movie_csv, df_movie_excel]:
        mask = (df['original_language'] == idioma) & (df['release_year'] == año)
        total += len(df[mask])
    
    return {"total_peliculas": total}

        
        
        
    

In [158]:
#Se genera una función que retorne en base al nombre de la película indique duración y fecha de lanzamiento
@app.get("/peliculas_duracion")
def peliculas_duracion(pelicula: str):
    for df in [df_movie1, df_movie_csv, df_movie_excel]:
        mask = df['original_title'] == pelicula
        pelicula_df = df[mask]
        if not pelicula_df.empty:
            duracion = pelicula_df['runtime'].values[0]
            año = pelicula_df['release_year'].values[0]
            return {"Duración": duracion, "Año": año}
    
    return {"message": "Película no encontrada"}


In [163]:
#Genero una función que en base a la franquicia perteneciente indique la cantidad de películas, ganancias totales
#y promedio

@app.get("/franquicia")
def franquicia(Franquicia: str):
    for df in [df_movie1, df_movie_csv, df_movie_excel]:
        mask = df['belongs_to_collection'].str.contains(Franquicia, na=False)
        franquicia_df = df[mask]
        if not franquicia_df.empty:
            cantidad_peliculas = len(franquicia_df)
            ganancia_total = franquicia_df['revenue'].sum()
            ganancia_promedio = franquicia_df['revenue'].mean()
            return {
                "Franquicia": Franquicia,
                "Cantidad de películas": cantidad_peliculas,
                "Ganancia total": ganancia_total,
                "Ganancia promedio": ganancia_promedio
            }
    
    return {"message": "Franquicia no encontrada"}


In [162]:
#Se realiza una función que según país indique la cantidad de peliculas producidas en el mismo.

@app.get("/peliculas_pais")
def peliculas_pais(Pais: str):
    for df in [df_movie1, df_movie_csv, df_movie_excel]:
        mask = df['production_countries'].str.contains(Pais, na=False)
        peliculas_pais_df = df[mask]
        if not peliculas_pais_df.empty:
            cantidad_peliculas = len(peliculas_pais_df)
            return {
                "País": Pais,
                "Cantidad de películas": cantidad_peliculas
            }
    
    return {"message": "País no encontrado"}

In [161]:
#Se ingresa el productor e indica  el revunue total y la cantidad de peliculas que realizo.
@app.get("/productoras_exitosas")
def productoras_exitosas(Productora: str):
    for df in [df_movie1, df_movie_csv, df_movie_excel]:
        mask = df['production_companies'].str.contains(Productora, na=False)
        productora_df = df[mask]
        if not productora_df.empty:
            cantidad_peliculas = len(productora_df)
            revenue_total = productora_df['revenue'].sum()
            return {
                "Productora": Productora,
                "Revenue total": revenue_total,
                "Cantidad de películas": cantidad_peliculas
            }
    
    return {"message": "Productora no encontrada"}

In [160]:
#Se genera una función en el cual se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo 
# devolver el éxito del mismo medido a través del retorno. 
# Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, 
# costo y ganancia de la misma, en formato lista.

@app.get("/get_director")
def get_director(nombre_director: str):
    director_movies = []
    for df in [df_movie1, df_movie_csv, df_movie_excel]:
        mask = df['director'].str.contains(nombre_director, na=False)
        director_df = df[mask]
        if not director_df.empty:
            movies = director_df[['original_title', 'release_date', 'revenue', 'budget']]
            movies = movies.rename(columns={'original_title': 'Título', 'release_date': 'Fecha de lanzamiento', 'revenue': 'Retorno', 'budget': 'Costo'})
            movies['Ganancia'] = movies['Retorno'] - movies['Costo']
            director_movies.append(movies)
    
    if director_movies:
        return {
            "Director": nombre_director,
            "Películas": director_movies
        }
    
    return {"message": "Director no encontrado"}

Git init + Instalaciones
Desde la misma terminal de VSCode, realizar los siguientes pasos:

git init
pip install uvicorn
pip install fastapi
Cualquier otra librería que se vaya a utilizar también puede ser descargada en este momento, o cuando sea necesario.

Pip freeze
Una vez que ya están todas las librerías descargadas en nuestro entorno virtual, podemos hacer el freeze de los requirements.
Es importante para este tutorial no abusar de demasiadas dependencias, porque en algunos casos, luego las aplicaciones no pueden deployar nuestros modelos.

pip freeze > requirements.txt